# How to setup Self-Hosted Runner in your github repository ?

Self Hosted runners are managed environments, there you can run all your workflows, when we say workflow it means
> .github/workflows/main.yaml  

in `main.yaml` we write jobs, which can be executed on the machine.

```
name: workflow

on:
    push:
        branches:
            - main
        paths-ignore:
            - 'README.md'

permissions:
    id-toke: write
    contents: read

jobs:
    integration:
        name: Continuous Integration
        runs-on: self-hosted
        steps:
            - name: Checkout Code
              uses: actions/checkout@v3

            - name: Lint code
              run: 1

            - name: Run unit tests
              run: echo "complete the github test"
```

if we go to the `main.yaml`, will write jobs here which can be executed on the machine which we are creating as self-hoted runner.

Basically we are creating a server using **EC2** instance in cloud, using self hoted runner we are commanding it when ever a commit is made to code, and all the configurations of the machine are managed by us.
all this is **Continuous Integration**

### Now, Let's see how we can configure it.

you can also follow this template [**github-actions-integration-template**](https://github.com/Rahul-404/github-actions-integration-template)

1. goto scripts

2. goto `self-hoted.sh`

```bash

## Docker install in ubuntu 22.04 lts
sudo apt-get update
sudo apt-get upgrade -y

sudo apt install apt-transport-https ca-cartificates curl software-properties-common -y
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo t

sudo apt update
apt-cache policy docker-ce
sudo apt install docker-ce -y
sudo systemctl status docker

# To install docker compose
sudo apt install docker-compose

# Add user into docker group
sudo usermod -aG docker ubuntu/cloud
newgrp docker

## Aws cli installation
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
sudo apt install unzip
unzip awscliv2.zip
sudo ./aws/install

## Github Runner configuration will start from here

## Download 
mkdir actions-runner && cd actions-runner
curl -o actions-runner-linux-x64-2.298.2.tar.gz -L https://github.com/actions/runner/releases/download/v2.298.2/actions-runner-linux-x64-2.298.2.tar.gz
echo "0bfd792196ce0ec6f1c65d2a9ad00215b2926ef2c416b8d97615265194477117  actions-runner-linux-x64-2.298.2.tar.gz" | shasum -a 256 -c
tar xzf ./actions-runner-linux-x64-2.298.2.tar.gz

## Configuration Important
./config.sh --url <url> --token <mention-you-token-here>
./run.sh

## Add Github runner as a service
sudo ./svc.sh install
sudo ./svc.sh start
sudo ./svc.sh status

## To stop the service
sudo ./svc.sh stop
sudo ./svc.sh uninstall
```

Go to **AWS EC2 Instance**  dashboard and start the already configured instance

![self-hosted](./images/self-hoted-runner/ec2-dashboard.png)

![self-hosted](./images/self-hoted-runner/self-hosted-1.png)

Once the instance is running, will be having *instance state* **✅ Running**

Now we can connect to the machine, click on the **connect** button

![self-hosted](./images/self-hoted-runner/self-hosted-2.png)

Again click on **connect**

![self-hosted](./images/self-hoted-runner/self-hosted-3.png)

Once we click on **connect** button ,a EC2 management console open up's.

![self-hosted](./images/self-hoted-runner/self-hosted-4.png)

Now clear the screen by typing **clear** in above instance screen

Now we have to do some configurations in the machine,

Goto your project and  goto your repository, click on $\text{Settings} \longrightarrow \text{Actions} \longrightarrow \text{Runner}$

then click on **New self-hosted runner**

![self-hosted](./images/self-hoted-runner/self-hosted-5.png)

you will reach to below page

![self-hosted](./images/self-hoted-runner/self-hosted-6.png)

in runner image select $Linux$

in Architecture select $x64$

and follow the following instructions, run those instruction one-by-one in our **EC2 Instance** terminal which we opened earlier.

**Download** insturction

![self-hosted](./images/self-hoted-runner/self-hosted-8.png)

**Configure** instruction

![self-hosted](./images/self-hoted-runner/self-hosted-9.png)

![self-hosted](./images/self-hoted-runner/self-hosted-10.png)

- We are using runner group **Default**

- For the runner name `self-hosted`

- Additional labels we are skipping by pressing **Enter**

![self-hosted](./images/self-hoted-runner/self-hosted-11.png)

now the final command `./run.sh`

![self-hosted](./images/self-hoted-runner/self-hosted-12.png)

Now **Github Actions** now configured, it's listening to the `Jobs`

![self-hosted](./images/self-hoted-runner/self-hosted-13.png)

Now **Runners** are *✅ Idle*

Now it requires my credential to access and do changes with trigger to images, so will authenticate with credentials

**Setup github secrets:**

```
AWS_ACCESS_KEY_TO = <aws-access-key>

AWC_SECRET_ACCESS_KEY = <aws-secret-access-key>

AWS_REGION = <aws-region>

AWS_ECR_LOGIN_URI = <aws-ecr-login-uri>

ECR_REPOSITORY_NAME = <name-of-repo> 
```



to add this secrets goto following path $\text{Settings} \longrightarrow \text{Secrets and variables} \longrightarrow \text{Actions} \longrightarrow \text{Repository secrets} \longrightarrow \text{New repository secret}$

![self-hosted](./images/self-hoted-runner/self-hosted-14.png)

Here add name of out secret in **Name** and secret key in **Secret** 

$Q.$ Where will get the secrets ?

$\longrightarrow$ we donloaded a one access key while creating the *IAM* user, there we have Access key (`AWS_ACCESS_KEY_TO`), and Secret access ID (`AWC_SECRET_ACCESS_KEY`)

Now aws region can be found on aws dashboard (`AWS_REGION`)

![self-hosted](./images/self-hoted-runner/self-hosted-15.png)

for `AWS_ECR_LOGIN_URI` use ECR repo link till <12-digit-number>.dkr.ecr.<region-name>.amazononaws.com skip the repository name i.e `production/customer-segmentation`.

![self-hosted](./images/self-hoted-runner/self-hosted-16.png)

now infally `ECR_REPOSITORY_NAME` it will the repository name here 'production/customer-segmentation'

Now you can start the deployment my making any commit to the repo, it will directly trigger the CI/CD pipeline and dockerize the project